In [1]:
import gradio as gr
import numpy as np
import cv2
import base64
from collections import Counter
import matplotlib.pyplot as plt
import tensorflow as tf

In [8]:
youtube_url="https://www.youtube.com/embed/XjhxRZEm638?rel=0"
youtube_url2="https://www.youtube.com/embed/NdW8Mh0lmOo"
youtube_url3="https://www.youtube.com/embed/u4UZ4UvZXrg"

embed_html1 = f"""
  <div style="position: relative; padding-bottom: 93%;">
    <iframe src="{youtube_url}" style="width: 100%; height: 100%; position: absolute; border: 0;" allowfullscreen scrolling="no" allow="accelerometer; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share;"></iframe>
  </div>"""
embed_html2= f"""<div style="position: relative; padding-bottom: 93%;">
    <iframe src="{youtube_url2}" style="width: 100%; height: 100%; position: absolute; border: 0;" allowfullscreen scrolling="no" allow="accelerometer; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share;"></iframe>
  </div>
  """
embed_html3= f"""
 <div style="position: relative; padding-bottom: 93%;">
    <iframe src="{youtube_url3}" style="width: 100%; height: 100%; position: absolute; border: 0;" allowfullscreen scrolling="no" allow="accelerometer; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share;"></iframe>
  </div>
 """
custom_css = """
body{
margin : 0px;
}
.gradio-container {
background-color: #092327 ;
margin:0px;
}
    .image-upload{
         display: flex;
        flex-direction: column;
        flex-grow: 1;
    }
     .image-upload2{
        flex-grow: 1;
    }
  
#Detection {
    background-color: #0B5351;
}

.svelte-1mhtq7j{
  background-color:#092327;
  background-image: linear-gradient(to bottom,#092327,#092327), linear-gradient(to bottom,#092327,#092327);
  background-clip: content-box, padding-box;

}
.svelte-1mhtq7j:hover{
  background-color:#00A9A5 ;
  background-image: linear-gradient(to bottom,#00A9A5,#00A9A5 ), linear-gradient(to bottom,#00A9A5,#00A9A5);
  background-clip: content-box, padding-box;

}
.svelte-1mhtq7j:checked {
  background-color:#00A9A5 ;
  background-image: linear-gradient(to bottom,#00A9A5,#00A9A5 ), linear-gradient(to bottom,#00A9A5,#00A9A5);
  background-clip: content-box, padding-box;

}
#Detection textarea{
    background-color: #00A9A5;
}

#Detection.changed textarea{
    background-color: #00A9A5;
}
#Detection.changed2 textarea{
    background-color: #D32F2F;
}
#Detection.changed2 {
    background-color: #D32F2F;
}
textarea {
    font-size: 20px;
    background-color:#092327;
}
.svelte-1gfkn6j{
font-size:20px;
}
#Detection label{
    font-size: 15px;
    font-weight: bold;
    background-color:#0B5351 ;
}
#Detection input[type=number]{
    text-size:20;
    background-color:#092327 ;
}
.svelte-1uw5tnk {
    font-size: 20px;
    font-weight: bold;
}
.lg secondary{
        display: block;
        margin: auto;
        font-size:15;
        background-color:#092327 !important;
    }
.svelte-cmf5ev{
        display: block;
        margin: auto;
        font-size:15;
        background-color:#092327 !important;
    }

}
.btn {
font-size: 18px !important;
    background-color: lightblue !important;
    color: darkblue !important;
    
    border: none;
    padding: 10px 20px;
    cursor: pointer;
}
"""


In [9]:

i=0
model= tf.keras.models.load_model("cnn_final_final.keras")
rep=[0]
fig=None
def Detection(input):
    global i,rep,fig
    if i%3==0:
        input_r =cv2.resize(input,(224,224))
        prec=rep[0]
        y_pred=model.predict(tf.keras.applications.mobilenet.preprocess_input(np.expand_dims(cv2.resize(input_r,(224,224)), axis=0)),verbose=0)
        rep=np.array(y_pred)[:,1]>0.8

        y_values =[np.array(y_pred)[:,0][0],np.array(y_pred)[:,1][0]]

        plt.figure(figsize=(5,3))#
        fig, ax = plt.subplots()
        ax.barh(y=["violence","non violence"], edgecolor='black',color=["#2a9d8f","#244454"],linewidth=1.5, alpha=0.9,width=y_values)
        ax.tick_params(axis='x', which='major', labelsize=12)
        ax.tick_params(axis='y', which='major', labelsize=12)
        ax.grid(True, which='both', linestyle='--', linewidth=0.3, color='gray', alpha=0.7)
        ax.set_axisbelow(True)


    
    if not(rep[0]==1):
        result=  str(i)+" normale"  
        audio_player = ''  
        img=None
        ch="changed"
    else:
        result= str(i)+" VIOLEEEEEEEEEEEENCE"
        ch="changed2"
        img=input
        screenshot=cv2.cvtColor(input, cv2.COLOR_BGR2RGB)
        cv2.imwrite("screenshots//"+str(i)+" violencee.png", screenshot) 
        audio_file=open("alarms//medical_alarm.mp3", 'rb')
        audio_content = audio_file.read()
        audio_base64 = base64.b64encode(audio_content).decode('utf-8')
        audio_player = f'<audio src="data:audio/mpeg;base64,{audio_base64}" autoplay controls></audio>'  
    
  
    i=i+1
    return gr.update(visible=True, value=result,elem_classes=ch),audio_player,img,fig

In [10]:
Hue=[21,40,75,155,190,270,295,315]
Sat=[0,0.2,0.7,1.1]

def Mat_adj(hists):
    n = len(hists)
    M = np.zeros((n, n), dtype=int)

    for i in range(n):
        M[i, :i] = np.linalg.norm(hists[i, None] - hists[:i], axis=1)

    return M 
def local_density(M):
    t=np.percentile(np.sort(M[M != 0.0].flatten()),2)

    locs=[]
    for i in range(len(M)):
        dist = np.concatenate((M[i+1:,i], M[i,:i]))
        loc = np.count_nonzero(dist <= t)
        locs.append(loc)
    return np.array(locs,dtype=np.float64)
def higher(M,locs):
    his=[]
    max_loc=np.max(locs)
    n=len(locs)
    for i in range(n):
        if locs[i]==max_loc:
            hi= np.max(np.concatenate((M[i+1:,i], M[i,:i])))
        else:
            indexes=   np.where(locs > locs[i])[0]
            hi=  np.min(np.concatenate((M[i,indexes[indexes < i]],M[indexes[indexes > i],i])))
        his.append(hi)
    return np.array(his,dtype=np.float64)
  

In [11]:

def Extract_key_frames(video_path,thr, Constraint):#
    imgs=[]
    frames=[]
    hists=[]
    imgs2=[]
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise Exception("Error opening video file.")      
    ret, frame = cap.read()
    if not ret:
        raise Exception("Error reading first frame from video.")  
    
    prev_frame = frame
    prev_frame=cv2.cvtColor(prev_frame,cv2.COLOR_BGR2RGB) 
    imgs.append(prev_frame)  
    ret, current_frame = cap.read()
    while ret:
        frames.append(cv2.cvtColor(current_frame,cv2.COLOR_BGR2RGB))
        diff = cv2.absdiff(current_frame, prev_frame)
        diff_count = np.count_nonzero(diff)    
        if diff_count >= thr: 
            current_frame=cv2.cvtColor(current_frame,cv2.COLOR_BGR2RGB)       
            imgs.append(current_frame)    
        prev_frame = current_frame
        ##
        
        current_frame = cv2.cvtColor(current_frame, cv2.COLOR_BGR2HSV)    
        h,s,v=cv2.split(current_frame)
        s= s.astype(np.float64) /255
        v=v.astype(np.float64)/255
        h,s,v=h.flatten(),s.flatten(),v.flatten()
        F=9*(np.digitize(h, Hue)%8)+3*(np.digitize(s,Sat)-1)+np.digitize(v,Sat)-1
        a=Counter(F)
        left=set(range(72) )- set(a.keys())
        a.update({x:0 for x in left})
        values_in_order = [a[key] for key in sorted(a.keys())]     
        hists.append(values_in_order)
        ##
        ret, current_frame= cap.read()
    hists=np.array(hists)
    cap.release()
    M=Mat_adj(hists)
    locs=local_density(M)
    his=higher(M,locs)
    locs/=np.max(locs)
    his/=np.max(his)
    both= np.column_stack([locs,his,np.arange(len(his), dtype=int)])
    both = both[both[:,1].argsort()[::-1]]
    _,idc=np.unique(both[:,0],return_index=True)
    both= both[idc]
    y=both[:,0]*both[:,1]
    g=sorted(y)
    y=y.tolist()
    liste= [ y.index(i) for i in g[-Constraint:] ]# not enought att
    a=np.array(sorted(both[liste,-1]),dtype=int)
    f=[frames[i] for i in a]
    for img in f:
        imgs2.append(img)
    return imgs,imgs2

In [12]:

font = cv2.FONT_HERSHEY_SIMPLEX 
org = (50, 50) 
fontScale = 1
color = (255, 255, 255) 
thickness = 2
def vd(video_path):#
    cap = cv2.VideoCapture(video_path)
    model= tf.keras.models.load_model("cnn_final_final.keras")
    ret, current_frame = cap.read()
    frame_cnt=0
    yield gr.update(visible=False),None, gr.update(visible=True),None,None,None
    
    while ret:
        current_frame=cv2.cvtColor(current_frame,cv2.COLOR_BGR2RGB)
        if frame_cnt%30==0:
            vd="non_violence"
            y_pred=model.predict(tf.keras.applications.mobilenet.preprocess_input(np.expand_dims(cv2.resize(current_frame,(224,224)), axis=0)),verbose=0)
            rep=np.array(y_pred)[:,1]>0.9
            audio_player = None
            fig=None
            ch="changed"
            plt.figure(figsize=(3,2))#
            fig, ax = plt.subplots()
            ax.barh(y=["normale","violence"], edgecolor='black',color=["#2a9d8f","#244454"],linewidth=1.5, alpha=0.9,width=[np.array(y_pred)[:,0][0],np.array(y_pred)[:,1][0]])
            plt.xlim(right=1)

            ax.grid(True, which='both', linestyle='--', linewidth=0.3, color='gray', alpha=0.7)
         
            screenshot=None
            if rep[0]==1:
                vd="violence"
                ch="changed2"
                screenshot=current_frame
                cv2.imwrite("screenshots//"+str(frame_cnt)+" violencee.png", screenshot) 
                audio_file=open("alarms//medical_alarm.mp3", 'rb')
                audio_content = audio_file.read()
                audio_base64 = base64.b64encode(audio_content).decode('utf-8')
                audio_player = f'<audio src="data:audio/mpeg;base64,{audio_base64}" autoplay controls></audio>'  
    
        yield  None,gr.update(visible=True, value=vd,elem_classes=ch) ,  current_frame,audio_player,fig,screenshot

        ret, current_frame= cap.read()
        
        frame_cnt+=1
    yield gr.update(visible=True),gr.update(visible=True, value="normale",elem_classes="changed"), gr.update(visible=False),None,None,None
 
    cap.release()
    cv2.destroyAllWindows()

In [13]:


def fn(j):
  if j=="CAMERA 4":
    return {htm1:gr.HTML(embed_html1,visible=True),htm2:gr.HTML(embed_html2,visible=False),htm3:gr.HTML(embed_html3,visible=False),input:gr.Image(sources=["webcam"], streaming=True,label="Camera de surveillance",visible=False),c1:gr.Column(scale=1,visible=True),c2:gr.Column(scale=1,visible=False)}
  elif j=="CAMERA 2":
    return {htm1:gr.HTML(embed_html1,visible=False),htm2:gr.HTML(embed_html2,visible=True),htm3:gr.HTML(embed_html3,visible=False),input:gr.Image(sources=["webcam"], streaming=True,label="Camera de surveillance",visible=False),c1:gr.Column(scale=1,visible=True),c2:gr.Column(scale=1,visible=False)}  
  elif j=="CAMERA 3":
    return {htm1:gr.HTML(embed_html1,visible=False),htm2:gr.HTML(embed_html2,visible=False),htm3:gr.HTML(embed_html3,visible=True),input:gr.Image(sources=["webcam"], streaming=True,label="Camera de surveillance",visible=False),c1:gr.Column(scale=1,visible=False),c2:gr.Column(scale=1,visible=True)}
  elif j=="CAMERA 1":
    return {htm1:gr.HTML(embed_html1,visible=False),htm2:gr.HTML(embed_html2,visible=False),htm3:gr.HTML(embed_html3,visible=False),input:gr.Image(sources=["webcam"], streaming=True,label="Camera de surveillance",visible=True),c1:gr.Column(scale=1,visible=False),c2:gr.Column(scale=1,visible=True)}
  else:
    return {htm1:gr.HTML(embed_html1,visible=True),htm2:gr.HTML(embed_html2,visible=True),htm3:gr.HTML(embed_html3,visible=True),input:gr.Image(sources=["webcam"], streaming=True,label="Camera de surveillance",visible=True),c1:gr.Column(scale=1,visible=True),c2:gr.Column(scale=1,visible=True)}

def fn2(j):
  if j=="CAMERA 4":
    return {htm1:gr.HTML(embed_html1,visible=True),htm2:gr.HTML(embed_html2,visible=False),htm3:gr.HTML(embed_html3,visible=False),inputvd:gr.Video(label="Camera de surveillance",visible=False),c1:gr.Column(scale=1,visible=True),c2:gr.Column(scale=1,visible=False)}
  elif j=="CAMERA 2":
    return {htm1:gr.HTML(embed_html1,visible=False),htm2:gr.HTML(embed_html2,visible=True),htm3:gr.HTML(embed_html3,visible=False),inputvd:gr.Video(label="Camera de surveillance",visible=False),c1:gr.Column(scale=1,visible=True),c2:gr.Column(scale=1,visible=False)}  
  elif j=="CAMERA 3":
    return {htm1:gr.HTML(embed_html1,visible=False),htm2:gr.HTML(embed_html2,visible=False),htm3:gr.HTML(embed_html3,visible=True),inputvd:gr.Video(label="Camera de surveillance",visible=False),c1:gr.Column(scale=1,visible=False),c2:gr.Column(scale=1,visible=True)}
  elif j=="CAMERA 1":
    return {htm1:gr.HTML(embed_html1,visible=False),htm2:gr.HTML(embed_html2,visible=False),htm3:gr.HTML(embed_html3,visible=False),inputvd:gr.Video(label="Camera de surveillance",visible=True),c1:gr.Column(scale=1,visible=False),c2:gr.Column(scale=1,visible=True)}
  else:
    return {htm1:gr.HTML(embed_html1,visible=True),htm2:gr.HTML(embed_html2,visible=True),htm3:gr.HTML(embed_html3,visible=True),inputvd:gr.Video(label="Camera de surveillance",visible=True),c1:gr.Column(scale=1,visible=True),c2:gr.Column(scale=1,visible=True)}


with gr.Blocks(css=custom_css) as demo:

  with gr.Tab(" SYSTEME DE DETECTION DE VIOLENCE"):
    with gr.Row(equal_height=True):
        with gr.Column(scale=1) as c2:  
          inputvd=gr.Video(sources='upload',include_audio=False,elem_id="Detection",elem_classes="image-upload2")
          oup=gr.Image(height=500,elem_id="Detection",visible=False,elem_classes="image-upload2")#
          # input=gr.Image(sources=["webcam"], streaming=True,label="Camera de surveillance",elem_id="Detection",elem_classes="image-upload2") 
          htm3=gr.HTML(embed_html3)
        with gr.Column(scale=1) as c1:
          htm1=gr.HTML(embed_html1)
          htm2=gr.HTML(embed_html2)
    
        with gr.Column(scale=0.5):
            output=gr.Textbox(label="ETAT",value="Non Violence",elem_id="Detection")
            probabilite=gr.Plot(elem_id="Detection")#gr.Textbox(label="prob",value="0")
            gr.Markdown(
      """
      ## CAPTURE DE VIOLENCE
      """)
            output3=gr.Image(height=200,elem_id="Detection")
            gr.Markdown(
      """
      ## ZOOM CAMERA
      """)
            with gr.Row():
              
              radio=gr.Radio(["CAMERA 1","CAMERA 2", "CAMERA 3","CAMERA 4","VUE GLOBALE"],elem_id="Detection")
              radio.change(fn=fn2, inputs=radio, outputs=[htm1,htm2,htm3,inputvd,c1,c2])
            examples = gr.Examples(examples=["DEMO/Fighting028_x264.mp4"],inputs=inputvd)#"DEMO/Normal_Videos_015_x264.mp4","
        output2 = gr.HTML()
        output2.visible = False
        inputvd.change(fn= vd,inputs= [inputvd],outputs=[inputvd,output,oup,output2,probabilite,output3])
        
        # gr.update(inputvd,visible=False)
        # gr.update(oup,visible=True)
  with gr.Tab("EXTRACTION D'IMAGE CLES"):

    with gr.Row(equal_height=True):
      
      with gr.Column():
        gr.Markdown(
    """
    # Video upload
    """)
        vido=gr.Video(sources='upload',include_audio=False,elem_id="Detection",autoplay=True)#,example=["C:/Users/mayah/Desktop/videos_test/a.mpg"]
        # slid= gr.Slider(1,5, value=5, step=1, label="Parcourir la video:")
     
        thr=gr.Number(label="Methode 1: Seuil",elem_id="Detection")
        constraint=gr.Number(label="Methode 2: Nombre à extraire",elem_id="Detection")
        # slid.change(slidee,slid,[])
        btn=gr.Button("GENERER",elem_classes="btn")
  
      with gr.Column():
        gr.Markdown(
    """
    # Images cles avec seuillage
    """)
        gallery = gr.Gallery(
            label="Generated images", show_label=False
        , columns=[3], rows=[4], object_fit="fit",height="290px",elem_id="Detection")
        gr.Markdown(
    """
    # Images cles avec clustering
    """)
        gallery2 = gr.Gallery(
            label="Generated images", show_label=False
        , columns=[3], rows=[4], object_fit="fit",height="290px",elem_id="Detection")
        btn.click(fn= Extract_key_frames,inputs= [vido,thr,constraint],outputs=[gallery,gallery2])
        examples = gr.Examples(examples=["DEMO/test2.mp4"],inputs=vido)
demo.queue()
demo.launch()

c:\Python311\Lib\site-packages\gradio\layouts\column.py:53: UserWarning: 'scale' value should be an integer. Using 0.5 will cause issues.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 4.19.1, however version 5.0.1 is available, please upgrade.
--------


Traceback (most recent call last):
  File "c:\Python311\Lib\site-packages\gradio\queueing.py", line 495, in call_prediction
    output = await route_utils.call_process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python311\Lib\site-packages\gradio\route_utils.py", line 233, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python311\Lib\site-packages\gradio\blocks.py", line 1617, in process_api
    data = await anyio.to_thread.run_sync(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python311\Lib\site-packages\anyio\to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python311\Lib\site-packages\anyio\_backends\_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
           ^^^^^^^^^^^^
  File "c:\Python311\Lib\site-packages\anyio\_backends\_asynci